In [222]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_friedman1
from sklearn.metrics import mean_squared_error
import sklearn.model_selection as ms

In [223]:
import numpy as np
np.random.seed(123)
import collections, copy, pickle
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [224]:
from mlxtend.frequent_patterns import apriori

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [225]:
import util.plot
import util.data

In [226]:
reload(util.plot)
reload(util.data);

In [227]:
with open('data/encoder.pkl', 'rb') as f:
    E = pickle.load(f)

In [228]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=1000)

In [229]:
for k in data.columns:
    if data[k].isna().sum() > 0:
        print('rm ', k)
        data.drop(columns=[k], inplace=True)

varsUsed = list(data.columns)
# varsUsed.remove('Unnamed')
varsUsed.remove('position')
varsUsed.remove('random_bool')
len(varsUsed)

rm  orig_destination_distance
rm  comp1_rate
rm  comp1_inv
rm  comp1_rate_percent_diff
rm  comp2_rate
rm  comp2_inv
rm  comp2_rate_percent_diff
rm  comp3_rate
rm  comp3_inv
rm  comp3_rate_percent_diff
rm  comp4_rate
rm  comp4_inv
rm  comp4_rate_percent_diff
rm  comp5_rate
rm  comp5_inv
rm  comp5_rate_percent_diff
rm  comp6_rate
rm  comp6_inv
rm  comp6_rate_percent_diff
rm  comp7_rate
rm  comp7_inv
rm  comp7_rate_percent_diff
rm  comp8_rate
rm  comp8_inv
rm  comp8_rate_percent_diff


49

In [230]:
E.discretizers

{'srch_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'site_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'visitor_location_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_country_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'prop_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_destination_id': KBinsDiscretizer(encode='ordinal', n_bins=array([9]), strategy='uniform'),
 'srch_adults_count': KBinsDiscretizer(encode='ordinal', n_bins=array([5]), strategy='uniform'),
 'srch_children_count': KBinsDiscretizer(encode='ordinal', n_bins=array([3]), strategy='uniform'),
 'srch_room_count': KBinsDiscretizer(encode='ordinal', n_bins=array([2]), strategy='uniform'),
 'position': KBinsDiscretizer(encode='ordinal', n_bins=array([38]), strategy='uniform'),
 'srch_length_of_stay': KBinsDiscretizer(encode='ordinal',

In [250]:
# TODO encode missing data, i.e. the hotels with no clicks

# note that even though the training score is an integer,
# the predicted score is a float. The remainder terms contain information about confidence.

y_labels = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'score']
y = data['score']
x = data[varsUsed].drop(columns=y_labels)
# x.shape, y.shape
# x_train, x_test, y_train, y_test = ms.train_test_split(x, y, test_size=0.3, random_state=42)
# pd.DataFrame.drop()
xy_train, xy_test = ms.train_test_split(data, random_state=42, test_size=0.3)
y = 'score'
y_train = xy_train[y].values
y_test = xy_test[y].values
x_train = xy_train.drop(columns=[y]).values
x_test = xy_test.drop(columns=[y]).values

y_train = y_train[:,np.newaxis]
y_test = y_test[:,np.newaxis]

In [251]:
x_train.shape, y_train.shape

((700, 50), (700, 1))

In [256]:
data

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,minute,weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,score
0,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,-0.216502,-0.209344,1,...,32,Thursday,0,0,0,0,1,0,0,0
1,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,0.814459,0.252784,1,...,32,Thursday,0,0,0,0,1,0,0,0
2,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,-0.216502,0.714912,1,...,32,Thursday,0,0,0,0,1,0,0,0
3,8.0,0.0,8.0,-0.961621,0.178773,0.0,0.0,-1.247462,0.252784,1,...,32,Thursday,0,0,0,0,1,0,0,0
4,8.0,0.0,8.0,-0.961621,0.178773,0.0,8.0,0.814459,-0.209344,1,...,32,Thursday,0,0,0,0,1,0,0,0
5,8.0,0.0,8.0,-0.961621,0.178773,0.0,8.0,0.814459,0.714912,1,...,32,Thursday,0,0,0,0,1,0,0,0
6,8.0,0.0,8.0,-0.961621,0.178773,0.0,8.0,-0.216502,-0.209344,1,...,32,Thursday,0,0,0,0,1,0,0,0
7,8.0,0.0,8.0,-0.961621,0.178773,0.0,8.0,-1.247462,-3.444240,0,...,32,Thursday,0,0,0,0,1,0,0,0
8,8.0,0.0,8.0,-0.961621,0.178773,0.0,8.0,0.814459,0.252784,1,...,32,Thursday,0,0,0,0,1,0,0,0
9,8.0,0.0,8.0,-0.961621,0.178773,0.0,8.0,0.814459,0.252784,1,...,32,Thursday,0,0,0,0,1,0,0,0


In [253]:
# models = {'GBoost':}
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
                                max_depth=2, random_state=0, loss='ls')
est.fit(x_train, y_train)
# mean_squared_error(y_test, est.predict(x_test))
# est.feature_importances_

# rfest = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
# rfest.fit(trainX[varsUsed], trainYM)
# rfest.feature_importances_
# mean_squared_error(testYM, rfest.predict(testX[varsUsed]))

ValueError: could not convert string to float: 'Wednesday'

In [21]:
# rfest.predict(trainX[varsUsed])[:100]
testYM

,click_bool,booking_bool
521,0,0
737,0,0
740,0,0
660,0,0
411,0,0
678,0,0
626,0,0
513,0,0
859,0,0
136,0,0
